# Libraries

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
# Load from an csv file
df = pd.read_csv('../data/fps-dataset-clean.csv', low_memory=False)
df

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,FPS
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2.0,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,70.0
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2.0,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,230.0
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,250.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,300.0
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,AMD Radeon R7 260X,GCN 2.0,104000.0,1100.0,1100.0,PCIe 3.0 x16,0.000160,12,1971000.0,128.0,2000.0,GDDR5,2.0,4.6,17600.0,28.0,16.0,6.3,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186031,AMD Ryzen 5 3600,6.0,12.0,100.0,576.0,3072.0,32.0,3600.0,36.0,1,7.0,65.0,4200.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2.0,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,51.5
186032,AMD Ryzen 5 2600X,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,1,12.0,95.0,4250.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2.0,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,49.9
186033,AMD Ryzen 5 1600X,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,1,14.0,95.0,4000.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2.0,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,46.9
186034,AMD Ryzen 5 2600,6.0,12.0,100.0,576.0,3072.0,16.0,3400.0,34.0,1,12.0,65.0,3900.0,AMD Radeon RX 570,GCN 4.0,224000.0,1168.0,1244.0,PCIe 3.0 x16,0.000232,12,5095000.0,256.0,4000.0,GDDR5,2.0,4.6,39810.0,14.0,32.0,6.4,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,46.4


In [4]:
# Showing basic information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186036 entries, 0 to 186035
Data columns (total 40 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CpuName                  186036 non-null  object 
 1   CpuNumberOfCores         186036 non-null  float64
 2   CpuNumberOfThreads       186036 non-null  float64
 3   CpuBaseClock             186036 non-null  float64
 4   CpuCacheL1               186036 non-null  float64
 5   CpuCacheL2               186036 non-null  float64
 6   CpuCacheL3               186036 non-null  float64
 7   CpuFrequency             186036 non-null  float64
 8   CpuMultiplier            186036 non-null  float64
 9   CpuMultiplierUnlocked    186036 non-null  int64  
 10  CpuProcessSize           186036 non-null  float64
 11  CpuTDP                   186036 non-null  float64
 12  CpuTurboClock            186036 non-null  float64
 13  GpuName                  186036 non-null  object 
 14  GpuA

# Feature Engineering

## André Pilco

### GPU to CPU ratio
In order to identify bottlenecks or imbalances, we calculate the ratio of GPU to CPU.

In [5]:
df['GpuCpuRatio'] = df.apply(lambda row: row['GpuBoostClock'] / row['CpuTurboClock'], axis = 1)

### Cache to Core ratio
Analyzing the ratio of CPU cache size (L1, L2 and L3) to the number of cores

In [6]:
df['CacheCoreRatio'] = df.apply(lambda row: (row['CpuCacheL1'] + row['CpuCacheL2'] + row['CpuCacheL3']) / row['CpuNumberOfCores'], axis = 1)

### Overclocking potential
Compute the potentital overclocking headrrom.

In [7]:
df['CpuOverclockPotential'] = df.apply(lambda row: (row['CpuTurboClock'] - row['CpuBaseClock']) / row['CpuBaseClock'], axis = 1)

## John Sovero

### EffectiveMemory
To determine how efficient a GPU is in terms of its computing capabilities

In [8]:
df['EffectiveMemory'] = df['GpuMemoryBus'] * df['GpuBandwidth']

### PerformanceClockRatio
To evaluate the ability of a CPU or GPU to handle high-performance tasks and to predict its behavior in high-demand situations.

In [9]:
df['PerformanceClockRatio'] = df['CpuTurboClock'] / df['GpuBoostClock']

### PerformancePerWatt
To capture how the most energy-efficient GPUs tend to maintain high performance per watt consumed

In [10]:
# GPU energy efficiency
df['PerformancePerWatt'] = df['GpuFP32Performance'] / df['GpuNumberOfTransistors']

## Iam Alvarez

### Ratio of Cores to CPU Threads
Measures the ratio between the number of physical cores and CPU threads to assess parallel processing capability.

In [11]:
df['CpuCoreThreadRatio'] = df['CpuNumberOfCores'] / df['CpuNumberOfThreads']

### CPU Power Index
It combines the turbo clock speed, the number of cores and the size of the L3 cache to evaluate the total CPU power.

In [12]:
df['CpuPowerIndex'] = df['CpuTurboClock'] * df['CpuNumberOfCores'] * df['CpuCacheL3']

### GPU Average Frequency Rating
Calculates the average GPU frequency to represent its overall operating performance.

In [13]:
df['GpuAverageClock'] = (df['GpuBaseClock'] + df['GpuBoostClock']) / 2

In [14]:
df.head()

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,FPS,GpuCpuRatio,CacheCoreRatio,CpuOverclockPotential,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2.0,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,70.0,0.431640,322.0,21.052632,65536000.0,2.316746,1023.508772,0.5,93856.0,1193.0
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2.0,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,230.0,0.431640,322.0,21.052632,65536000.0,2.316746,1023.508772,0.5,93856.0,1193.0
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,250.0,0.456053,321.5,37.000000,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,300.0,0.456053,321.5,37.000000,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,AMD Radeon R7 260X,GCN 2.0,104000.0,1100.0,1100.0,PCIe 3.0 x16,0.000160,12,1971000.0,128.0,2000.0,GDDR5,2.0,4.6,17600.0,28.0,16.0,6.3,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,200.0,0.289474,321.5,37.000000,13312000.0,3.454545,947.596154,1.0,91200.0,1100.0


### Clean Dataset

In [15]:
# We drop duplicated rows
print(df[df.duplicated()].shape)
df.drop_duplicates(inplace=True)

(8, 49)


In [16]:
# Find rows where Cpu name are equal and Gpu name are equal and Gamesetting are equal
testing = df[df.duplicated(subset=['CpuName', 'GpuName', 'GameName', 'GameSetting'], keep=False)]
testing[(testing['CpuName'] == 'Intel Core i5-4690') & (testing['GpuName'] == 'NVIDIA GeForce GTX 1080 11Gbps') & (testing['GameName'] == 'counterStrikeGlobalOffensive')]

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,FPS,GpuCpuRatio,CacheCoreRatio,CpuOverclockPotential,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,250.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,300.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
14100,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,high,220.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
14101,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,high,250.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
14102,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,high,270.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0
14103,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,high,300.0,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0


Nos damos cuenta que existen CpuName, GpuName y Game iguales pero con FPS distintos. Así que sacamos el promedio de FPS para ponerlo en una sola fila, ya que las demás variables dependen de estas 3 características.

In [17]:
# Creating the new dataset with mean FPS in rows that have the same values in the columns
columns_to_check = df.columns.tolist()
columns_to_check.remove('FPS')
df = df.groupby(columns_to_check, as_index=False).agg({'FPS': 'mean'})
print(df.shape)
df.head()

(93695, 49)


,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,GpuCpuRatio,CacheCoreRatio,CpuOverclockPotential,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock,FPS
0,AMD Athlon 200GE,2.0,4.0,100.0,192.0,1024.0,4.0,3200.0,32.0,0,14.0,35.0,3200.0,AMD Radeon 550,GCN 4.0,56000.0,1100.0,1183.0,PCIe 3.0 x8,0.000103,12,1211000.0,64.0,2000.0,GDDR5,2.0,4.6,18930.0,14.0,16.0,6.4,512.0,32.0,37860.0,2200.0,1.2.131,fortnite,1080.0,med,0.369688,610.0,31.0,3584000.0,2.704987,550.454545,0.5,25600.0,1141.5,120.0
1,AMD Athlon 200GE,2.0,4.0,100.0,192.0,1024.0,4.0,3200.0,32.0,0,14.0,35.0,3200.0,AMD Radeon 550,GCN 4.0,56000.0,1100.0,1183.0,PCIe 3.0 x8,0.000103,12,1211000.0,64.0,2000.0,GDDR5,2.0,4.6,18930.0,14.0,16.0,6.4,512.0,32.0,37860.0,2200.0,1.2.131,minecraft,720.0,high,0.369688,610.0,31.0,3584000.0,2.704987,550.454545,0.5,25600.0,1141.5,120.0
2,AMD Athlon 200GE,2.0,4.0,100.0,192.0,1024.0,4.0,3200.0,32.0,0,14.0,35.0,3200.0,AMD Radeon HD 7750,GCN 1.0,72000.0,800.0,800.0,PCIe 3.0 x16,0.000123,12,819200.0,128.0,1024.0,GDDR5,1.2,4.6,12800.0,28.0,16.0,5.1,512.0,32.0,25600.0,1500.0,1.2.131,grandTheftAuto5,720.0,med,0.250000,610.0,31.0,9216000.0,4.000000,546.133333,0.5,25600.0,800.0,50.0
3,AMD Athlon 200GE,2.0,4.0,100.0,192.0,1024.0,4.0,3200.0,32.0,0,14.0,35.0,3200.0,AMD Radeon HD 7850,GCN 1.0,153600.0,860.0,860.0,PCIe 3.0 x16,0.000212,12,1761000.0,256.0,2000.0,GDDR5,1.2,4.6,27520.0,28.0,32.0,5.1,1024.0,64.0,55040.0,2800.0,1.2.131,apexLegends,720.0,low,0.268750,610.0,31.0,39321600.0,3.720930,628.928571,0.5,25600.0,860.0,30.0
4,AMD Athlon 200GE,2.0,4.0,100.0,192.0,1024.0,4.0,3200.0,32.0,0,14.0,35.0,3200.0,AMD Radeon HD 7850,GCN 1.0,153600.0,860.0,860.0,PCIe 3.0 x16,0.000212,12,1761000.0,256.0,2000.0,GDDR5,1.2,4.6,27520.0,28.0,32.0,5.1,1024.0,64.0,55040.0,2800.0,1.2.131,apexLegends,720.0,med,0.268750,610.0,31.0,39321600.0,3.720930,628.928571,0.5,25600.0,860.0,30.0


In [18]:
# Verifying that is correct the formula above
df[(df['CpuName'] == 'Intel Core i7-920') & (df['GpuName'] == 'AMD Radeon RX 480') & (df['GameName'] == 'counterStrikeGlobalOffensive')]

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,GpuCpuRatio,CacheCoreRatio,CpuOverclockPotential,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock,FPS
86506,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,0,45.0,130.0,2933.0,AMD Radeon RX 480,GCN 4.0,256000.0,1120.0,1266.0,PCIe 3.0 x16,0.000232,12,5834000.0,256.0,8000.0,GDDR5,2.0,4.6,40510.0,14.0,32.0,6.4,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,0.43164,322.0,21.052632,65536000.0,2.316746,1023.508772,0.5,93856.0,1193.0,150.0


In [19]:
# Veryfing with another data
df[(df['CpuName'] == 'Intel Core i5-4690') & (df['GpuName'] == 'NVIDIA GeForce GTX 1080 11Gbps') & (df['GameName'] == 'counterStrikeGlobalOffensive')]

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,GpuCpuRatio,CacheCoreRatio,CpuOverclockPotential,EffectiveMemory,PerformanceClockRatio,PerformancePerWatt,CpuCoreThreadRatio,CpuPowerIndex,GpuAverageClock,FPS
51729,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,high,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0,260.0
51730,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,0,22.0,84.0,3800.0,NVIDIA GeForce GTX 1080 11Gbps,Pascal,352300.0,1607.0,1733.0,PCIe 3.0 x16,0.000314,12,8873000.0,256.0,8000.0,GDDR5X,1.2,4.6,110900.0,16.0,64.0,6.4,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,0.456053,321.5,37.0,90188800.0,2.192729,1232.361111,1.0,91200.0,1670.0,275.0


In [20]:
# Save file with new features
df.to_csv('../data/fps-dataset-feature.csv', index=False)